## Importing Data and Libraries

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.4.1
Running on TPU  ['10.82.148.234:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.82.148.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.82.148.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import os
import numpy as np
import string
from tqdm.notebook import tqdm
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Sequential, callbacks, utils
from sklearn.model_selection import train_test_split

In [ ]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

## Preprocessing

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def tokenize(data, to_select_lines=-1):

  result = []
  table = str.maketrans('', '', string.punctuation+'|.।')

  idx = 0

  for line in data:
    line_tokens = []
    idx += 1
    if (to_select_lines == -1):
      idx -= 1
    if (idx == to_select_lines):
      break

    for word in line.split():
      if len(word.translate(table)) > 2:
        line_tokens.append(word.translate(table))

    result.append(line_tokens)

  return result

In [ ]:
def create_dict(tokens, to_select_words=-1):

  word_dict = {}
  for i in tokens:
    for j in i:
      try:
        word_dict[j] += 1
      except:
        word_dict[j] = 1

  res_dict = {k: v for k, v in sorted(word_dict.items(), key=lambda item: item[1], reverse=True)}


  if (to_select_words == -1):
    final_dict = {}
    for idx, (key, value) in zip(range(len(res_dict)), res_dict.items()):
      final_dict[key] = idx + 1
    return final_dict

  final_dict = {}
  for idx, (key, value) in zip(range(to_select_words), res_dict.items()):
    final_dict[key] = idx + 1

  return final_dict

In [ ]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)

  return result

In [ ]:
def get_sequences(data, word_dict, n_gram_length):

  input_sequence = []
  idx = 0

  for line in data:
    token_list = convert(line, word_dict)
    for i in range(1, len(token_list)):
      n_gram_seq = token_list[:i+1]
      if not (n_gram_seq[-1] == 0 or sum(n_gram_seq) - n_gram_seq[-1] == 0):
        input_sequence.append(n_gram_seq)

  input_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence, n_gram_length, padding='pre')

  return input_sequence

In [ ]:
def preprocess(data, to_select_lines=-1, to_select_words=-1, n_gram_length=3):

  print("Tokenizing")
  tokenized = tokenize(data, to_select_lines)
  print("Creating Dictionary")
  word_dict = create_dict(tokenized, to_select_words)
  print("Creating Input Sequences")
  input_sequences = get_sequences(tokenized, word_dict, n_gram_length)

  del tokenized

  return word_dict, input_sequences

## Changing data for input

In [ ]:
## Splitting the data for train and test
data = np.array(data)
train, valid = train_test_split(data, test_size=0.05, random_state=1234)
train.shape, valid.shape

((94836,), (4992,))

In [ ]:
word_dict, train_sequences = preprocess(train, to_select_words=10000, n_gram_length=3)

Tokenizing
Creating Dictionary
Creating Input Sequences


In [ ]:
train_sequences[:10]

array([[   0,  337,  133],
       [ 337,  133,   12],
       [ 133,   12, 8390],
       [  12, 8390,    2],
       [8390,    2,  174],
       [   2,  174, 2906],
       [ 174, 2906,    2],
       [2906,    2, 1868],
       [1868,    0,    2],
       [   0,    2,  685]], dtype=int32)

In [ ]:
valid_tokens = tokenize(valid)
valid_sequences = get_sequences(valid_tokens, word_dict, n_gram_length=3)

In [ ]:
valid_sequences[:10]

array([[   0,  922, 2876],
       [ 922, 2876,    3],
       [2876,    3, 1775],
       [   3, 1775,  700],
       [1775,  700, 7058],
       [ 700, 7058, 5315],
       [   0, 9607,   16],
       [  16,    0,    3],
       [   0,    3, 2789],
       [   3, 2789, 1131]], dtype=int32)

In [ ]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [ ]:
len(train_sequences), len(valid_sequences), len(word_dict)

(1238642, 64306, 10000)

In [ ]:
train_sequences = np.array(train_sequences)
np.random.shuffle(train_sequences)
train_sequences[:3]

array([[ 209, 1394,  395],
       [   0,   53, 1622],
       [2986, 4589,   22]], dtype=int32)

In [ ]:
train_sequences.shape, len(word_dict)

((1238642, 3), 10000)

In [ ]:
def split_x_label(train):
  x = train[:-1]
  label = train[-1]
  label = tf.one_hot(label, len(word_dict))
  return x, label

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(train_sequences)

In [ ]:
dataset = dataset.map(split_x_label)

In [ ]:
dataset = dataset.batch(256, drop_remainder=True)

In [ ]:
for i,j in dataset:
  print(i.shape, j.shape)
  break

(256, 2) (256, 10000)


## Saving the dictionary and sequence

In [ ]:
a = pd.DataFrame(data = word_dict.keys())
a['values'] = word_dict.values()
a.columns = ['words', 'values']
a.head()
a.to_csv('/content/drive/MyDrive/Next word Generation/Dictionary/word_dict_entry_1.csv', index=False)

In [ ]:
np.save('/content/drive/MyDrive/Next word Generation/Sequences/seq_train_entry_1.npy', train_sequences)
np.save('/content/drive/MyDrive/Next word Generation/Sequences/seq_valid_entry_1.npy', valid_sequences)

## Creating Model

In [ ]:
def create_model(word_dict):

  model = Sequential([
                    layers.Embedding(len(word_dict)+1, 64),
                    layers.Bidirectional(layers.LSTM(512)),
                    layers.Dense(len(word_dict), activation='softmax')
          ])
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

## Training the model

In [ ]:
def check_results(input_string, model, word_dict, reverse_dict, generation_len=10):

  test_input = input_string
  next_words = generation_len
  table = str.maketrans('', '', string.punctuation)

  for i in range(next_words):
    token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
    token_list = convert(token_list, word_dict)
    token_list = [token_list[-2:]]
    predicted =  model.predict_classes(token_list, verbose=0)[0]
    
    output_word = ''
    if predicted in reverse_dict:
      output_word = reverse_dict[predicted]
    test_input += " " + output_word

    print(test_input)

In [ ]:
model = create_model(word_dict)

In [ ]:
save_point = callbacks.ModelCheckpoint('/content/drive/MyDrive/Next word Generation/Models/model_entry_1.h5', monitor='loss', save_best_only=True)
stop_check = callbacks.EarlyStopping(
                monitor='loss', min_delta=0, patience=4, verbose=0,
                mode='auto', baseline=None, restore_best_weights=False
              )

In [ ]:
model.fit(dataset.repeat(), epochs=20, steps_per_epoch=5000, callbacks=[save_point])

Epoch 1/20
5000/5000 [==============================] - 282s 56ms/step - loss: 2.1340 - accuracy: 0.4955
Epoch 2/20
5000/5000 [==============================] - 283s 57ms/step - loss: 2.0942 - accuracy: 0.5023
Epoch 3/20
5000/5000 [==============================] - 284s 57ms/step - loss: 2.0775 - accuracy: 0.5053
Epoch 4/20
5000/5000 [==============================] - 284s 57ms/step - loss: 2.0619 - accuracy: 0.5078
Epoch 5/20
5000/5000 [==============================] - 282s 56ms/step - loss: 2.0479 - accuracy: 0.5101
Epoch 6/20
5000/5000 [==============================] - 283s 57ms/step - loss: 2.0332 - accuracy: 0.5125
Epoch 7/20
5000/5000 [==============================] - 283s 57ms/step - loss: 2.0210 - accuracy: 0.5142
Epoch 8/20
5000/5000 [==============================] - 282s 56ms/step - loss: 2.0095 - accuracy: 0.5160
Epoch 9/20
5000/5000 [==============================] - 283s 57ms/step - loss: 1.9994 - accuracy: 0.5176
Epoch 10/20
5000/5000 [==============================] 

## Functions for Evaluating the model

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Next word Generation/Models/model_entry_1.h5')

In [ ]:
valid_sequence = np.load('/content/drive/MyDrive/Next word Generation/Sequences/seq_valid_entry_1.npy')

In [ ]:
def calculate_perplexity(predictions, labels):

  log_probabilities = []
  for i in range(0, len(predictions)):
    log_prob = np.log(np.sum(predictions[i] * labels[i]))
    log_probabilities.append(log_prob)

  sum_log_probabilities = np.sum(log_probabilities)
  per_char_log_probability = sum_log_probabilities / len(predictions)
  perplexity = 1/np.exp(per_char_log_probability)
  return perplexity

## Checking results

In [ ]:
len(valid_sequence)

64306

In [ ]:
valid_sequence = valid_sequence[:100]

In [ ]:
valid_sequence[:10]

array([[   0,  922, 2876],
       [ 922, 2876,    3],
       [2876,    3, 1775],
       [   3, 1775,  700],
       [1775,  700, 7058],
       [ 700, 7058, 5315],
       [   0, 9607,   16],
       [  16,    0,    3],
       [   0,    3, 2789],
       [   3, 2789, 1131]], dtype=int32)

In [ ]:
valid_label = model.predict(valid_sequence[:, :-1])

In [ ]:
calculate_perplexity(valid_label, valid_sequence[:, -1])

0.004532609215547525

## Visualizing the results

In [ ]:
path = '/content/drive/MyDrive/Next word Generation/Data/train.txt'
data = ''
with open(path, 'r', encoding='utf-8') as file:
  data = file.readlines()

In [ ]:
data[50:55]

["ਇਸ ਤੋਂ ਪਹਿਲਾਂ ਸਨਿਚਰਵਾਰ ਭਾਰਤ ਸਰਕਾਰ ਨੇ ਕਿਹਾ ਕਿ ਉਹ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕਾਂ ਵਲੋਂ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਨੂੰ ਮਾਰ ਕੇ ਉਨ੍ਹਾਂ ਦਾ ਸਿਰ ਕਲਮ ਕਰਕੇ ਲਿਜਾਣ ਵਿਰੁੱਧ ਬਦਲੇ ਦੀ ਕਰਵਾਈ ਨਹੀਂ ਕਰੇਗਾ ਪਰ ਏਅਰ ਚੀਫ ਮਾਰਸ਼ਲ ਨਾਰਮਨ ਅਨਿਲ ਕੁਮਾਰ ਬਰਾਉਨ ਨੇ ਕਲ੍ਹ ਕਿਹਾ ਸੀ ਕਿ 2003 ਤੋਂ ਕੰਟਰੋਲ ਰੇਖਾ 'ਤੇ ਚਲੀ ਆ ਰਹੀ ਜੰਗਬੰਦੀ ਦੀ ਪਾਕਿ ਵਲੋਂ ਉਲੰਘਣਾ ਕਦੇ ਵੀ ਬਰਦਾਸ਼ਤ ਨਹੀਂ ਕੀਤੀ ਜਵੇਗੀ |\n",
 "ਅਸੀਂ ਸਥਿਤੀ 'ਤੇ ਸਾਵਧਾਨੀ ਨਾਲ ਨਜ਼ਰ ਰੱਖ ਰਹੇ ਹਾਂ |\n",
 'ਜੇਕਰ ਉਲੰਘਣਾ ਜਾਰੀ ਰਾਹੀ ਤਾਂ ਅਸੀਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਸੋਚਾਂਗੇ ਪਰ ਉਨ੍ਹਾਂ ਦੂਸਰੇ ਬਦਲ ਬਾਰੇ ਦੱਸਣ ਤੋਂ ਇਨਕਾਰ ਕਰ ਦਿੱਤਾ ਸੀ |\n',
 "ਜੰਮੂ ਤੇ ਕਸ਼ਮੀਰ ਵਿਚ ਭਾਰਤੀ ਫ਼ੌਜ ਵਲੋਂ ਕਥਿਤ ਰੂਪ ਵਿਚ ਕੀਤੀ ਗੋਲੀਬਾਰੀ ਵਿਚ ਪਾਕਿਸਤਾਨੀ ਸੈਨਿਕ ਦੇ ਮਾਰੇ ਜਣ ਪਿੱਛੋਂ 6 ਜਨਵਰੀ ਤੋਂ ਹੀ ਭਾਰਤ ਅਤੇ ਪਾਕਿ ਵਿਚਕਾਰ ਸਰਹੱਦ 'ਤੇ ਤਣਾਅ ਬਣਿਆਂ ਹੋਇਆ ਹੈ |\n",
 'ਦੋ ਦਿਨ ਪਿੱਛੋਂ ਪਾਕਿ ਸੈਨਿਕ ਪੁਣਛ ਜ਼ਿਲ੍ਹੇ ਵਿਚ ਮੇਂਧਰ ਸੈਕਟਰ ਵਿਚ ਦੋ ਭਾਰਤੀ ਸੈਨਿਕਾਂ ਦਾ ਸਿਰ ਵੱਢ ਕੇ ਲੈ ਗਏ |\n']

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Next word Generation/Dictionary/word_dict_entry_1.csv')
model = tf.keras.models.load_model('/content/drive/MyDrive/Next word Generation/Models/model_entry_1.h5')

In [ ]:
word_dict = {}
for i,j in zip(df['words'], df['values']):
  word_dict[i] = j

In [ ]:
reverse_dict = {}
for key, value in word_dict.items():
  reverse_dict[value] = key

In [ ]:
table = str.maketrans('', '', string.punctuation+'|.।')

In [ ]:
def convert(line, word_dict):
  result = []
  for word in line:
    try:
      result.append(word_dict[word])
    except:
      result.append(0)
  return result

In [ ]:
test_input = 'ਮੇਰੀ ਇੰਡੀਆ'
next_words = 10
array = np.zeros((10, len(word_dict)))

for i in range(next_words):

  token_list = [w.translate(table) for w in test_input.split() if w.translate(table) != '']
  token_list = convert(token_list, word_dict)
  token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], 6, padding='pre')
  print(token_list)

  array[i] = model.predict(token_list, verbose=0)[0]

  values = np.argsort(array[i])

  pred_words = []
  for x,y in zip(range(5), values[::-1]):
    pred_words.append(reverse_dict[y])
    print(x, "  ", reverse_dict[y], " ", array[i][y])

  selected = int(input('Enter the index of the selected word.\n'))

  if selected == 0:
    test_input += " " + pred_words[0]
  elif selected == 1:
    test_input += " " + pred_words[1]
  elif selected == 2:
    test_input += " " + pred_words[2]
  elif selected == 3:
    test_input += " " + pred_words[3]
  elif selected == 4:
    test_input += " " + pred_words[4]


  print(test_input)

In [ ]:
reverse_dict[1113]

In [ ]:
array[0, 1113]

In [ ]:
array[0]

In [ ]:
sorted(array[1], reverse=True)

In [ ]:
np.argsort(array[0])

In [ ]:
array.shape

In [ ]:
array[1]